# ML Models
#### Using Main branch past preprocessed data

In [28]:
import os
root_dir = '../../' if os.getcwd().split('/')[-1] != 'credit-now' else './'
os.chdir(root_dir)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



from sklearn import model_selection 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from xgboost import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


import warnings
warnings.filterwarnings("ignore")  


import credit_data
from tqdm import tqdm
from itertools import combinations
from joblib import dump, load

In [29]:
os.getcwd()

'/Users/seunguk/Documents/likelion/credit-now'

In [30]:
# train_data, test_data, train_label, test_label = credit_data.load_train_data(test_size = 0.3, encoding = True)
# print(train_data.shape, test_data.shape, train_label.shape, test_label.shape)

float형 열들을 전부 int형으로 변환. <br>
<br>
train_test_split()과 encoding을 전부 적용하지 않은 원본 데이터프레임에서 <br>
<br>
numeric 열을 전부 범위로 감싸서 categorical 열로 만들고 one-hot encoding 적용 <br>

#### 해당 내용 잘 안됨

In [31]:
train_data, train_label = credit_data.load_train_data(test_size = 0, encoding = False)
print(train_data.shape, train_label.shape)

(26457, 26) (26457, 2)


In [32]:
del train_label['index']

In [33]:
train_label

,credit
0,1.0
1,1.0
2,2.0
3,0.0
4,2.0
...,...
26452,1.0
26453,2.0
26454,2.0
26455,2.0


In [34]:
train_data

,index,age,begin_month,car,career,cluster,days_income,days_unemployed,edu_type,email,...,income_type,month_birth,month_unemployed,occyp_type,phone,reality,week_birth,week_unemployed,work_phone,credit
0,0,38,6.0,1,12,30,10.882416,9190,1,1,...,1,7.0,6.0,1,1,1,1.0,0.0,1,1.0
1,1,31,5.0,1,4,15,19.156347,9840,2,2,...,1,7.0,4.0,2,1,2,1.0,1.0,1,1.0
2,2,52,22.0,2,12,8,19.131840,14653,1,1,...,2,0.0,8.0,3,2,2,2.0,1.0,1,2.0
3,3,41,37.0,1,5,30,11.786962,12996,2,1,...,1,10.0,1.0,4,2,2,3.0,0.0,1,0.0
4,4,41,26.0,2,5,5,9.187959,12932,1,1,...,3,9.0,11.0,3,1,2,0.0,3.0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,33,2.0,1,5,26,15.999431,10095,2,1,...,3,6.0,0.0,6,1,1,1.0,2.0,1,1.0
26453,26453,41,47.0,1,6,23,10.131712,12816,1,1,...,2,5.0,7.0,1,1,2,0.0,2.0,1,2.0
26454,26454,27,25.0,2,5,10,24.179549,8067,2,1,...,2,0.0,4.0,6,1,1,0.0,0.0,1,2.0
26455,26455,27,59.0,1,0,33,16.679672,10038,3,1,...,2,2.0,10.0,2,1,2,1.0,2.0,1,2.0


In [35]:
train_data.info() 
# begin_month, days_income, family_size, income_per, income_total,
# month_birth, month_unemployed, week_birth, week_unemployed,
# credit

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             26457 non-null  int64  
 1   age               26457 non-null  int64  
 2   begin_month       26457 non-null  float64
 3   car               26457 non-null  int64  
 4   career            26457 non-null  int64  
 5   cluster           26457 non-null  int64  
 6   days_income       26457 non-null  float64
 7   days_unemployed   26457 non-null  int64  
 8   edu_type          26457 non-null  int64  
 9   email             26457 non-null  int64  
 10  family_size       26457 non-null  float64
 11  family_type       26457 non-null  int64  
 12  gender            26457 non-null  int64  
 13  house_type        26457 non-null  int64  
 14  income_per        26457 non-null  float64
 15  income_total      26457 non-null  float64
 16  income_type       26457 non-null  int64 

In [36]:
train_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   credit  26457 non-null  float64
dtypes: float64(1)
memory usage: 206.8 KB


In [37]:
# begin_month, days_income, family_size, income_per, income_total,
# month_birth, month_unemployed, week_birth, week_unemployed,
# credit

In [38]:
# train_data.begin_month = train_data.begin_month.astype(int) 
# train_data.days_income = train_data.days_income.astype(int) 
# train_data.family_size = train_data.family_size.astype(int)
# train_data.income_per = train_data.income_per.astype(int)  
# train_data.income_total = train_data.income_total.astype(int) 
# train_data.month_birth = train_data.month_birth.astype(int) 
# train_data.month_unemployed = train_data.month_unemployed.astype(int) 
# train_data.week_birth = train_data.week_birth.astype(int) 
# train_data.week_unemployed = train_data.week_unemployed.astype(int) 


# train_label.credit = train_label.credit.astype(int) 

One-Hot Encoding

In [39]:
# def make_pipeline(df: pd.DataFrame, num_features: list, cat_features: list):
#     data = df.drop(['index','credit'], axis=1).copy()
#     num_features = [feature for feature in num_features if feature not in ['index','credit']]
#     cat_features = [feature for feature in cat_features if feature not in ['index','credit']]

#     numerical_transformer = StandardScaler()
#     categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore')

#     preprocessor = ColumnTransformer(
#         transformers=[
#             ('num', numerical_transformer, num_features),
#             ('cat', categorical_transformer, cat_features)])

#     pipe = Pipeline(steps=[('preprocessor', preprocessor)])
#     pipe.fit(data)

In [40]:
# begin_month, days_income, family_size, income_per, income_total,
# month_birth, month_unemployed, week_birth, week_unemployed,
# credit

In [41]:
# num_features = ['begin_month, days_income,income_per, income_total,month_birth, month_unemployed, week_birth, week_unemployed']
# cat_features = ['family_size']

In [42]:
# make_pipeline(train_data, num_features, cat_features)

---

# train_old, train 불러와서 처리

In [43]:
train_old = pd.read_csv('credit_data/train_old.csv')
train_new = pd.read_csv('credit_data/train.csv')


In [44]:
train_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22724 entries, 0 to 22723
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          22724 non-null  int64  
 1   gender         22724 non-null  int64  
 2   car            22724 non-null  int64  
 3   reality        22724 non-null  int64  
 4   child_num      22724 non-null  int64  
 5   income_total   22724 non-null  float64
 6   income_type    22724 non-null  int64  
 7   edu_type       22724 non-null  int64  
 8   family_type    22724 non-null  int64  
 9   house_type     22724 non-null  int64  
 10  age            22724 non-null  float64
 11  employed_year  22724 non-null  float64
 12  work_phone     22724 non-null  int64  
 13  phone          22724 non-null  int64  
 14  email          22724 non-null  int64  
 15  occyp_type     22724 non-null  int64  
 16  family_size    22724 non-null  int64  
 17  begin_year     22724 non-null  float64
 18  credit

In [45]:
train_new

,index,age,begin_month,car,career,cluster,days_income,days_unemployed,edu_type,email,...,income_type,month_birth,month_unemployed,occyp_type,phone,reality,week_birth,week_unemployed,work_phone,credit
0,0,38,6.0,1,12,30,10.882416,9190,1,1,...,1,7.0,6.0,1,1,1,1.0,0.0,1,1.0
1,1,31,5.0,1,4,15,19.156347,9840,2,2,...,1,7.0,4.0,2,1,2,1.0,1.0,1,1.0
2,2,52,22.0,2,12,8,19.131840,14653,1,1,...,2,0.0,8.0,3,2,2,2.0,1.0,1,2.0
3,3,41,37.0,1,5,30,11.786962,12996,2,1,...,1,10.0,1.0,4,2,2,3.0,0.0,1,0.0
4,4,41,26.0,2,5,5,9.187959,12932,1,1,...,3,9.0,11.0,3,1,2,0.0,3.0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,33,2.0,1,5,26,15.999431,10095,2,1,...,3,6.0,0.0,6,1,1,1.0,2.0,1,1.0
26453,26453,41,47.0,1,6,23,10.131712,12816,1,1,...,2,5.0,7.0,1,1,2,0.0,2.0,1,2.0
26454,26454,27,25.0,2,5,10,24.179549,8067,2,1,...,2,0.0,4.0,6,1,1,0.0,0.0,1,2.0
26455,26455,27,59.0,1,0,33,16.679672,10038,3,1,...,2,2.0,10.0,2,1,2,1.0,2.0,1,2.0


In [46]:
train_old.income_total = train_old.income_total.astype(int) 
train_old.age = train_old.age.astype(int) 
train_old.begin_year = train_old.begin_year.astype(int)
train_old.employed_year = train_old.employed_year.astype(int) 

train_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22724 entries, 0 to 22723
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   index          22724 non-null  int64
 1   gender         22724 non-null  int64
 2   car            22724 non-null  int64
 3   reality        22724 non-null  int64
 4   child_num      22724 non-null  int64
 5   income_total   22724 non-null  int64
 6   income_type    22724 non-null  int64
 7   edu_type       22724 non-null  int64
 8   family_type    22724 non-null  int64
 9   house_type     22724 non-null  int64
 10  age            22724 non-null  int64
 11  employed_year  22724 non-null  int64
 12  work_phone     22724 non-null  int64
 13  phone          22724 non-null  int64
 14  email          22724 non-null  int64
 15  occyp_type     22724 non-null  int64
 16  family_size    22724 non-null  int64
 17  begin_year     22724 non-null  int64
 18  credit         22724 non-null  int64
dtypes: i

In [47]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             26457 non-null  int64  
 1   age               26457 non-null  int64  
 2   begin_month       26457 non-null  float64
 3   car               26457 non-null  int64  
 4   career            26457 non-null  int64  
 5   cluster           26457 non-null  int64  
 6   days_income       26457 non-null  float64
 7   days_unemployed   26457 non-null  int64  
 8   edu_type          26457 non-null  int64  
 9   email             26457 non-null  int64  
 10  family_size       26457 non-null  float64
 11  family_type       26457 non-null  int64  
 12  gender            26457 non-null  int64  
 13  house_type        26457 non-null  int64  
 14  income_per        26457 non-null  float64
 15  income_total      26457 non-null  float64
 16  income_type       26457 non-null  int64 

In [48]:
train_new.begin_month = train_new.begin_month.astype(int) 
train_new.days_income = train_new.days_income.astype(int) 
train_new.family_size = train_new.family_size.astype(int)
train_new.income_per = train_new.income_per.astype(int) 
train_new.income_total = train_new.income_total.astype(int) 
train_new.month_birth = train_new.month_birth.astype(int) 
train_new.month_unemployed = train_new.month_unemployed.astype(int)
train_new.week_birth = train_new.week_birth.astype(int)
train_new.week_unemployed = train_new.week_unemployed.astype(int)
train_new.credit = train_new.credit.astype(int)


train_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   index             26457 non-null  int64
 1   age               26457 non-null  int64
 2   begin_month       26457 non-null  int64
 3   car               26457 non-null  int64
 4   career            26457 non-null  int64
 5   cluster           26457 non-null  int64
 6   days_income       26457 non-null  int64
 7   days_unemployed   26457 non-null  int64
 8   edu_type          26457 non-null  int64
 9   email             26457 non-null  int64
 10  family_size       26457 non-null  int64
 11  family_type       26457 non-null  int64
 12  gender            26457 non-null  int64
 13  house_type        26457 non-null  int64
 14  income_per        26457 non-null  int64
 15  income_total      26457 non-null  int64
 16  income_type       26457 non-null  int64
 17  month_birth       26457 non-nul

In [49]:
train_new 

,index,age,begin_month,car,career,cluster,days_income,days_unemployed,edu_type,email,...,income_type,month_birth,month_unemployed,occyp_type,phone,reality,week_birth,week_unemployed,work_phone,credit
0,0,38,6,1,12,30,10,9190,1,1,...,1,7,6,1,1,1,1,0,1,1
1,1,31,5,1,4,15,19,9840,2,2,...,1,7,4,2,1,2,1,1,1,1
2,2,52,22,2,12,8,19,14653,1,1,...,2,0,8,3,2,2,2,1,1,2
3,3,41,37,1,5,30,11,12996,2,1,...,1,10,1,4,2,2,3,0,1,0
4,4,41,26,2,5,5,9,12932,1,1,...,3,9,11,3,1,2,0,3,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,33,2,1,5,26,15,10095,2,1,...,3,6,0,6,1,1,1,2,1,1
26453,26453,41,47,1,6,23,10,12816,1,1,...,2,5,7,1,1,2,0,2,1,2
26454,26454,27,25,2,5,10,24,8067,2,1,...,2,0,4,6,1,1,0,0,1,2
26455,26455,27,59,1,0,33,16,10038,3,1,...,2,2,10,2,1,2,1,2,1,2


In [50]:
old_data = train_old.copy()
old_label = train_old[['index', 'credit']]

old_data = old_data.drop(['index', 'credit'], axis=1)
old_label = np.array(old_label[['credit']])
old_train, old_test, old_train_label, old_test_label = \
            model_selection.train_test_split(old_data, old_label, test_size=0.3,
                                            random_state=0, stratify=old_label) 

new_data = train_new.copy() 
new_label = train_new[['index', 'credit']]  

new_data = new_data.drop(['index', 'credit'], axis = 1)
new_label = np.array(new_label[['credit']]) 
new_train, new_test, new_train_label, new_test_label = \
             model_selection.train_test_split(new_data, new_label, test_size=0.3,
                                            random_state=0, stratify=new_label) 


---

## NGBoost

In [51]:
#!pip install --upgrade git+https://github.com/stanfordmlgroup/ngboost.git

In [52]:
#### NGBoost
from ngboost import NGBClassifier
from ngboost.distns import k_categorical, Bernoulli

ngb_cat = NGBClassifier(Dist=k_categorical(3), verbose=False) # tell ngboost that there are 3 possible outcomes


Compare old & new

In [53]:
_ = ngb_cat.fit(old_train, old_train_label) # Y should have only 3 values: {0,1,2} 


ngb_pred_test = ngb_cat.predict(old_test)

accuracy_score(ngb_pred_test, old_test_label)

0.6406570841889117

In [54]:
_ = ngb_cat.fit(new_train, new_train_label) # Y should have only 3 values: {0,1,2} 


ngb_pred_test = ngb_cat.predict(new_test)

accuracy_score(ngb_pred_test, new_test_label)

0.6924918115394306

In [55]:
# print(round(accuracy_score(ngb_cat.predict(train_data), train_label), 3))

In [56]:
# ngb_cat.predict(test_data)

In [57]:
# ngb_cat.predict_proba(test_data)

In [62]:
ngb_cat.pred_dist(new_test).params

{'p0': array([0.14151951, 0.13084966, 0.1195992 , ..., 0.29091686, 0.11412647,
        0.10206274]),
 'p1': array([0.25072773, 0.14621426, 0.16669859, ..., 0.65859704, 0.22018458,
        0.15644116]),
 'p2': array([0.60775276, 0.72293607, 0.71370221, ..., 0.05048611, 0.66568895,
        0.7414961 ])}

In [ ]:
ngb_cat.pred_dist(new_test).params

In [ ]:
#### Scores 
from ngboost.scores import LogScore, CRPScore 

NGBClassifier(Dist = k_categorical(3), Score = LogScore, verbose=False).fit(train_data, train_label)

NGBClassifier(random_state=RandomState(MT19937) at 0x103E7FB40, verbose=False)